# EMSN AtmosBird Cloud Classifier

**Doel:** Train een CNN model om bewolkingsgraad te classificeren uit hemelbeelden.

**Dataset:** AtmosBird camera beelden van Pi Berging (Nijverdal)

**Output:** ONNX model voor deployment op Raspberry Pi

---

## Workflow
1. Upload beelden naar Colab (of mount Google Drive)
2. Label beelden met interactieve tool
3. Train EfficientNet-B0 met transfer learning
4. Exporteer naar ONNX voor Pi deployment

In [ ]:
# @title 1. Setup & Dependencies
# Installeer benodigde packages
!pip install -q torch torchvision timm onnx onnxruntime pillow matplotlib scikit-learn tqdm ipywidgets

import os
import json
import random
import shutil
from pathlib import Path
from datetime import datetime
from typing import List, Tuple, Dict, Optional

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# @title 2. Mount Google Drive (optioneel)
# Als je beelden via Google Drive wilt laden

USE_GDRIVE = True  # @param {type:"boolean"}

if USE_GDRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Maak werkdirectory
    WORK_DIR = Path('/content/drive/MyDrive/EMSN/cloud_classifier')
    WORK_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Werkdirectory: {WORK_DIR}")
else:
    WORK_DIR = Path('/content/cloud_classifier')
    WORK_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Lokale werkdirectory: {WORK_DIR}")

## 3. Beelden Uploaden

**Optie A:** Upload een ZIP bestand met beelden

**Optie B:** Kopieer beelden naar Google Drive folder

De beelden moeten JPG formaat zijn. Ideaal: 200-500 beelden met variatie in:
- Helder (clear sky)
- Bewolkt (overcast)
- Gedeeltelijk bewolkt (partly cloudy)
- Dag en nacht

In [ ]:
# @title 3a. Upload ZIP bestand met beelden
from google.colab import files
import zipfile

UPLOAD_ZIP = False  # @param {type:"boolean"}

if UPLOAD_ZIP:
    print("Upload een ZIP bestand met hemelbeelden...")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            print(f"Uitpakken: {filename}")
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall(WORK_DIR / 'raw_images')
            print(f"Klaar! Beelden staan in {WORK_DIR / 'raw_images'}")

In [ ]:
# @title 3b. Of specificeer pad naar beelden in Google Drive
# Pas dit pad aan naar waar je beelden staan

IMAGE_SOURCE_DIR = "/content/drive/MyDrive/EMSN/atmosbird_images"  # @param {type:"string"}

# Zoek alle JPG beelden
source_path = Path(IMAGE_SOURCE_DIR)
if source_path.exists():
    image_files = list(source_path.glob('**/*.jpg')) + list(source_path.glob('**/*.jpeg'))
    print(f"Gevonden: {len(image_files)} beelden")
    
    # Kopieer naar werkdirectory (subset voor sneller werken)
    MAX_IMAGES = 500  # @param {type:"integer"}
    
    raw_dir = WORK_DIR / 'raw_images'
    raw_dir.mkdir(exist_ok=True)
    
    # Selecteer willekeurige subset
    if len(image_files) > MAX_IMAGES:
        selected = random.sample(image_files, MAX_IMAGES)
    else:
        selected = image_files
    
    print(f"Kopieren van {len(selected)} beelden...")
    for img_path in tqdm(selected):
        dest = raw_dir / img_path.name
        if not dest.exists():
            shutil.copy(img_path, dest)
    
    print(f"Klaar! {len(list(raw_dir.glob('*.jpg')))} beelden in {raw_dir}")
else:
    print(f"Pad niet gevonden: {IMAGE_SOURCE_DIR}")
    print("Upload beelden via sectie 3a of pas het pad aan.")

In [ ]:
# @title 4. Interactieve Labeling Tool
import ipywidgets as widgets
from IPython.display import display, clear_output

class CloudLabeler:
    """
    Interactieve tool om hemelbeelden te labelen.
    
    Klassen:
    - helder: Blauwe lucht, weinig tot geen wolken (< 20%)
    - gedeeltelijk: Mix van wolken en blauwe lucht (20-80%)
    - bewolkt: Volledig of grotendeels bewolkt (> 80%)
    - nacht_helder: Nachtelijke hemel, sterren zichtbaar
    - nacht_bewolkt: Nachtelijke hemel, geen sterren zichtbaar
    """
    
    CLASSES = ['helder', 'gedeeltelijk', 'bewolkt', 'nacht_helder', 'nacht_bewolkt', 'overslaan']
    CLASS_COLORS = {
        'helder': '#3498db',
        'gedeeltelijk': '#f39c12', 
        'bewolkt': '#95a5a6',
        'nacht_helder': '#2c3e50',
        'nacht_bewolkt': '#1a1a2e',
        'overslaan': '#e74c3c'
    }
    
    def __init__(self, image_dir: Path, labels_file: Path):
        self.image_dir = Path(image_dir)
        self.labels_file = Path(labels_file)
        self.labels = self._load_labels()
        self.images = self._get_unlabeled_images()
        self.current_idx = 0
        
        # UI elementen
        self.output = widgets.Output()
        self.progress = widgets.IntProgress(
            value=len(self.labels),
            min=0,
            max=len(self.images) + len(self.labels),
            description='Voortgang:'
        )
        self.status = widgets.Label(value=f"{len(self.labels)} gelabeld, {len(self.images)} te gaan")
        
    def _load_labels(self) -> Dict[str, str]:
        if self.labels_file.exists():
            with open(self.labels_file, 'r') as f:
                return json.load(f)
        return {}
    
    def _save_labels(self):
        with open(self.labels_file, 'w') as f:
            json.dump(self.labels, f, indent=2)
    
    def _get_unlabeled_images(self) -> List[Path]:
        all_images = list(self.image_dir.glob('*.jpg')) + list(self.image_dir.glob('*.jpeg'))
        return [img for img in all_images if img.name not in self.labels]
    
    def _show_image(self, img_path: Path):
        with self.output:
            clear_output(wait=True)
            
            # Laad en resize beeld
            img = Image.open(img_path)
            img.thumbnail((800, 600))
            
            fig, ax = plt.subplots(figsize=(12, 8))
            ax.imshow(img)
            ax.set_title(f"{img_path.name}\n({self.current_idx + 1}/{len(self.images)})", fontsize=14)
            ax.axis('off')
            plt.tight_layout()
            plt.show()
    
    def _on_button_click(self, label: str):
        def handler(b):
            if self.current_idx < len(self.images):
                img_name = self.images[self.current_idx].name
                
                if label != 'overslaan':
                    self.labels[img_name] = label
                    self._save_labels()
                
                self.current_idx += 1
                self.progress.value = len(self.labels)
                self.status.value = f"{len(self.labels)} gelabeld, {len(self.images) - self.current_idx} te gaan"
                
                if self.current_idx < len(self.images):
                    self._show_image(self.images[self.current_idx])
                else:
                    with self.output:
                        clear_output(wait=True)
                        print("Klaar met labelen!")
                        print(f"Totaal gelabeld: {len(self.labels)} beelden")
                        self._print_stats()
        return handler
    
    def _print_stats(self):
        print("\nVerdeling per klasse:")
        for cls in self.CLASSES[:-1]:  # Zonder 'overslaan'
            count = sum(1 for v in self.labels.values() if v == cls)
            print(f"  {cls}: {count}")
    
    def start(self):
        """Start de labeling interface."""
        if not self.images:
            print("Alle beelden zijn al gelabeld!")
            self._print_stats()
            return
        
        # Maak knoppen
        buttons = []
        for cls in self.CLASSES:
            btn = widgets.Button(
                description=cls.replace('_', ' ').title(),
                button_style='info' if cls != 'overslaan' else 'danger',
                layout=widgets.Layout(width='150px', height='40px')
            )
            btn.on_click(self._on_button_click(cls))
            buttons.append(btn)
        
        # Layout
        button_row = widgets.HBox(buttons)
        
        print("Klik op de juiste categorie voor elk beeld:")
        print("- Helder: blauwe lucht, < 20% wolken")
        print("- Gedeeltelijk: mix, 20-80% wolken") 
        print("- Bewolkt: > 80% wolken")
        print("- Nacht Helder: sterren zichtbaar")
        print("- Nacht Bewolkt: geen sterren")
        print("")
        
        display(self.progress)
        display(self.status)
        display(button_row)
        display(self.output)
        
        # Toon eerste beeld
        self._show_image(self.images[self.current_idx])

In [ ]:
# @title 5. Start Labeling
# Pas paden aan indien nodig

RAW_IMAGES_DIR = WORK_DIR / 'raw_images'
LABELS_FILE = WORK_DIR / 'labels.json'

if RAW_IMAGES_DIR.exists():
    labeler = CloudLabeler(RAW_IMAGES_DIR, LABELS_FILE)
    labeler.start()
else:
    print(f"Geen beelden gevonden in {RAW_IMAGES_DIR}")
    print("Upload eerst beelden via sectie 3.")

In [ ]:
# @title 6. Bekijk Label Statistieken
if LABELS_FILE.exists():
    with open(LABELS_FILE, 'r') as f:
        labels = json.load(f)
    
    print(f"Totaal gelabeld: {len(labels)} beelden\n")
    
    # Tel per klasse
    class_counts = {}
    for label in labels.values():
        class_counts[label] = class_counts.get(label, 0) + 1
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 5))
    classes = list(class_counts.keys())
    counts = list(class_counts.values())
    colors = [CloudLabeler.CLASS_COLORS.get(c, '#333') for c in classes]
    
    ax.bar(classes, counts, color=colors)
    ax.set_xlabel('Klasse')
    ax.set_ylabel('Aantal beelden')
    ax.set_title('Verdeling gelabelde beelden')
    
    for i, (c, v) in enumerate(zip(classes, counts)):
        ax.text(i, v + 1, str(v), ha='center')
    
    plt.tight_layout()
    plt.show()
else:
    print("Nog geen labels. Start eerst de labeling tool.")

---
## Model Training

Nu we gelabelde data hebben, kunnen we het model trainen.

In [ ]:
# @title 7. Dataset Klasse

class CloudDataset(Dataset):
    """PyTorch Dataset voor cloud classificatie."""
    
    # Vereenvoudigde klassen voor model (dag/nacht samen)
    CLASS_MAPPING = {
        'helder': 0,
        'nacht_helder': 0,  # Samen met helder
        'gedeeltelijk': 1,
        'bewolkt': 2,
        'nacht_bewolkt': 2  # Samen met bewolkt
    }
    CLASS_NAMES = ['helder', 'gedeeltelijk', 'bewolkt']
    
    def __init__(self, image_dir: Path, labels: Dict[str, str], transform=None):
        self.image_dir = Path(image_dir)
        self.transform = transform
        
        # Filter alleen gelabelde beelden (zonder 'overslaan')
        self.samples = []
        for img_name, label in labels.items():
            if label in self.CLASS_MAPPING:
                img_path = self.image_dir / img_name
                if img_path.exists():
                    self.samples.append((img_path, self.CLASS_MAPPING[label]))
        
        print(f"Dataset: {len(self.samples)} beelden")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Laad beeld
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
# @title 8. Model Configuratie

# Hyperparameters
IMAGE_SIZE = 224  # EfficientNet input size
BATCH_SIZE = 32  # @param {type:"integer"}
EPOCHS = 20  # @param {type:"integer"}
LEARNING_RATE = 0.001  # @param {type:"number"}
NUM_CLASSES = 3

# Data augmentation voor training
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE + 32, IMAGE_SIZE + 32)),
    transforms.RandomCrop(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validatie transform (geen augmentation)
val_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print(f"Image size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")

In [ ]:
# @title 9. Laad Data en Maak Train/Val Split

# Laad labels
with open(LABELS_FILE, 'r') as f:
    all_labels = json.load(f)

# Filter bruikbare labels
usable_labels = {k: v for k, v in all_labels.items() if v in CloudDataset.CLASS_MAPPING}
print(f"Bruikbare gelabelde beelden: {len(usable_labels)}")

# Split in train/val (80/20)
items = list(usable_labels.items())
train_items, val_items = train_test_split(items, test_size=0.2, random_state=42)

train_labels = dict(train_items)
val_labels = dict(val_items)

print(f"Training: {len(train_labels)} beelden")
print(f"Validatie: {len(val_labels)} beelden")

# Maak datasets
train_dataset = CloudDataset(RAW_IMAGES_DIR, train_labels, transform=train_transform)
val_dataset = CloudDataset(RAW_IMAGES_DIR, val_labels, transform=val_transform)

# Maak dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"\nTrain batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

In [ ]:
# @title 10. Maak Model (EfficientNet-B0 met Transfer Learning)

class CloudClassifier(nn.Module):
    """Cloud classifier gebaseerd op EfficientNet-B0."""
    
    def __init__(self, num_classes=3, pretrained=True):
        super().__init__()
        
        # Laad pretrained EfficientNet-B0
        self.backbone = timm.create_model(
            'efficientnet_b0',
            pretrained=pretrained,
            num_classes=0  # Verwijder classifier
        )
        
        # Bepaal feature dimensie
        self.feature_dim = self.backbone.num_features
        
        # Nieuwe classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.feature_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        features = self.backbone(x)
        return self.classifier(features)
    
    def get_cloud_coverage(self, x):
        """Bereken bewolkingspercentage uit model output."""
        with torch.no_grad():
            logits = self.forward(x)
            probs = torch.softmax(logits, dim=1)
            
            # Gewogen som: helder=0%, gedeeltelijk=50%, bewolkt=100%
            weights = torch.tensor([0.0, 0.5, 1.0], device=x.device)
            coverage = (probs * weights).sum(dim=1) * 100
            
            return coverage, probs

# Maak model
model = CloudClassifier(num_classes=NUM_CLASSES).to(device)

# Tel parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model parameters: {total_params:,} (trainable: {trainable_params:,})")

In [ ]:
# @title 11. Training Loop

# Loss en optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

best_val_acc = 0.0
best_model_state = None

print("Start training...\n")

for epoch in range(EPOCHS):
    # Training
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
    
    train_loss /= train_total
    train_acc = train_correct / train_total
    
    # Validatie
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
    
    val_loss /= val_total
    val_acc = val_correct / val_total
    
    # Update scheduler
    scheduler.step()
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
        print(f"  -> Nieuw beste model! Val acc: {val_acc:.4f}")
    
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, "
          f"Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

print(f"\nTraining klaar! Beste validatie accuracy: {best_val_acc:.4f}")

In [ ]:
# @title 12. Plot Training History

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1.plot(history['train_loss'], label='Train Loss')
ax1.plot(history['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training & Validation Loss')
ax1.legend()
ax1.grid(True)

# Accuracy
ax2.plot(history['train_acc'], label='Train Acc')
ax2.plot(history['val_acc'], label='Val Acc')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training & Validation Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig(WORK_DIR / 'training_history.png', dpi=150)
plt.show()

In [ ]:
# @title 13. Evaluatie op Validatieset

# Laad beste model
model.load_state_dict(best_model_state)
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

# Classification report
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=CloudDataset.CLASS_NAMES))

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks(range(len(CloudDataset.CLASS_NAMES)))
ax.set_yticks(range(len(CloudDataset.CLASS_NAMES)))
ax.set_xticklabels(CloudDataset.CLASS_NAMES)
ax.set_yticklabels(CloudDataset.CLASS_NAMES)
ax.set_xlabel('Voorspeld')
ax.set_ylabel('Werkelijk')
ax.set_title('Confusion Matrix')

# Annotaties
for i in range(len(CloudDataset.CLASS_NAMES)):
    for j in range(len(CloudDataset.CLASS_NAMES)):
        ax.text(j, i, str(cm[i, j]), ha='center', va='center', 
                color='white' if cm[i, j] > cm.max()/2 else 'black')

plt.colorbar(im)
plt.tight_layout()
plt.savefig(WORK_DIR / 'confusion_matrix.png', dpi=150)
plt.show()

In [ ]:
# @title 14. Export naar ONNX (voor Pi deployment)

# Laad beste model
model.load_state_dict(best_model_state)
model.eval()

# Dummy input voor export
dummy_input = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE).to(device)

# Export pad
ONNX_PATH = WORK_DIR / 'cloud_classifier.onnx'
PT_PATH = WORK_DIR / 'cloud_classifier.pt'

# Export naar PyTorch
torch.save({
    'model_state_dict': model.state_dict(),
    'class_names': CloudDataset.CLASS_NAMES,
    'image_size': IMAGE_SIZE,
    'best_val_acc': best_val_acc,
    'training_date': datetime.now().isoformat()
}, PT_PATH)
print(f"PyTorch model opgeslagen: {PT_PATH}")

# Export naar ONNX
torch.onnx.export(
    model.cpu(),
    dummy_input.cpu(),
    ONNX_PATH,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

print(f"ONNX model opgeslagen: {ONNX_PATH}")
print(f"Model grootte: {ONNX_PATH.stat().st_size / 1024 / 1024:.1f} MB")

In [ ]:
# @title 15. Valideer ONNX Model
import onnx
import onnxruntime as ort

# Laad en valideer ONNX model
onnx_model = onnx.load(ONNX_PATH)
onnx.checker.check_model(onnx_model)
print("ONNX model is valide!")

# Test inference met ONNX Runtime
ort_session = ort.InferenceSession(str(ONNX_PATH))

# Test met dummy input
test_input = np.random.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE).astype(np.float32)
outputs = ort_session.run(None, {'input': test_input})

print(f"\nONNX inference test:")
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {outputs[0].shape}")
print(f"Output (logits): {outputs[0]}")

# Softmax naar probabilities
probs = np.exp(outputs[0]) / np.exp(outputs[0]).sum()
print(f"Probabilities: {probs}")
print(f"Predicted class: {CloudDataset.CLASS_NAMES[np.argmax(probs)]}")

In [ ]:
# @title 16. Download Modellen
from google.colab import files

print("Download de modellen voor deployment op de Pi:")
print(f"\n1. ONNX model (aanbevolen voor Pi): {ONNX_PATH.name}")
print(f"2. PyTorch checkpoint: {PT_PATH.name}")
print(f"3. Labels bestand: {LABELS_FILE.name}")

# Download knoppen
if ONNX_PATH.exists():
    files.download(str(ONNX_PATH))

# Of kopieer naar Google Drive
if USE_GDRIVE:
    print(f"\nBestanden staan ook in Google Drive: {WORK_DIR}")

---
## Pi Deployment

Kopieer het ONNX model naar de Pi en gebruik onderstaande code voor inference.

In [ ]:
# @title 17. Pi Inference Code (kopieer naar Pi)

PI_INFERENCE_CODE = '''
#!/usr/bin/env python3
"""
Cloud Classifier Inference voor Raspberry Pi
Gebruik het getrainde ONNX model voor bewolkingsdetectie.

Installatie:
    pip install onnxruntime pillow numpy

Gebruik:
    from cloud_classifier_inference import CloudClassifierONNX
    
    classifier = CloudClassifierONNX('/path/to/cloud_classifier.onnx')
    result = classifier.predict('/path/to/sky_image.jpg')
    print(f"Bewolking: {result['cloud_coverage_percent']:.1f}%")
"""

import numpy as np
from PIL import Image
import onnxruntime as ort
from pathlib import Path
from typing import Dict, Union


class CloudClassifierONNX:
    """ONNX-based cloud classifier voor Raspberry Pi."""
    
    CLASS_NAMES = ['helder', 'gedeeltelijk', 'bewolkt']
    IMAGE_SIZE = 224
    
    # ImageNet normalization
    MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    
    def __init__(self, model_path: Union[str, Path]):
        """Laad ONNX model."""
        self.model_path = Path(model_path)
        if not self.model_path.exists():
            raise FileNotFoundError(f"Model niet gevonden: {self.model_path}")
        
        # Laad ONNX model
        self.session = ort.InferenceSession(
            str(self.model_path),
            providers=['CPUExecutionProvider']  # Pi heeft geen CUDA
        )
        self.input_name = self.session.get_inputs()[0].name
    
    def preprocess(self, image_path: Union[str, Path]) -> np.ndarray:
        """Preprocess beeld voor model input."""
        # Laad en resize
        img = Image.open(image_path).convert('RGB')
        img = img.resize((self.IMAGE_SIZE, self.IMAGE_SIZE), Image.BILINEAR)
        
        # Naar numpy array [H, W, C] -> [C, H, W]
        img_array = np.array(img, dtype=np.float32) / 255.0
        img_array = (img_array - self.MEAN) / self.STD
        img_array = np.transpose(img_array, (2, 0, 1))  # HWC -> CHW
        
        # Voeg batch dimensie toe [1, C, H, W]
        return np.expand_dims(img_array, 0)
    
    def predict(self, image_path: Union[str, Path]) -> Dict:
        """
        Voorspel bewolkingsklasse en percentage.
        
        Returns:
            dict met:
                - class_name: voorspelde klasse
                - class_index: klasse index (0-2)
                - probabilities: dict met kans per klasse
                - cloud_coverage_percent: geschat bewolkingspercentage (0-100)
                - confidence: hoogste waarschijnlijkheid
        """
        # Preprocess
        input_tensor = self.preprocess(image_path)
        
        # Inference
        outputs = self.session.run(None, {self.input_name: input_tensor})
        logits = outputs[0][0]  # [num_classes]
        
        # Softmax
        exp_logits = np.exp(logits - np.max(logits))  # Numeriek stabiel
        probs = exp_logits / exp_logits.sum()
        
        # Resultaat
        class_idx = int(np.argmax(probs))
        
        # Bereken bewolkingspercentage (gewogen som)
        # helder=0%, gedeeltelijk=50%, bewolkt=100%
        weights = np.array([0.0, 50.0, 100.0])
        cloud_coverage = float(np.sum(probs * weights))
        
        return {
            'class_name': self.CLASS_NAMES[class_idx],
            'class_index': class_idx,
            'probabilities': {
                name: float(prob) for name, prob in zip(self.CLASS_NAMES, probs)
            },
            'cloud_coverage_percent': cloud_coverage,
            'confidence': float(probs[class_idx])
        }


# Test code
if __name__ == "__main__":
    import sys
    
    if len(sys.argv) < 3:
        print("Gebruik: python cloud_classifier_inference.py <model.onnx> <image.jpg>")
        sys.exit(1)
    
    model_path = sys.argv[1]
    image_path = sys.argv[2]
    
    classifier = CloudClassifierONNX(model_path)
    result = classifier.predict(image_path)
    
    print(f"Beeld: {image_path}")
    print(f"Klasse: {result['class_name']}")
    print(f"Bewolking: {result['cloud_coverage_percent']:.1f}%")
    print(f"Confidence: {result['confidence']:.2%}")
    print(f"Probabilities: {result['probabilities']}")
'''

# Sla inference code op
inference_path = WORK_DIR / 'cloud_classifier_inference.py'
with open(inference_path, 'w') as f:
    f.write(PI_INFERENCE_CODE)

print(f"Pi inference code opgeslagen: {inference_path}")
print("\nKopieer dit bestand samen met het ONNX model naar de Pi.")

---
## Klaar!

Je hebt nu:
1. **labels.json** - Je gelabelde dataset
2. **cloud_classifier.onnx** - Getraind model voor Pi
3. **cloud_classifier.pt** - PyTorch checkpoint (backup)
4. **cloud_classifier_inference.py** - Inference code voor Pi

### Deployment op Pi Berging

```bash
# Kopieer bestanden naar Pi
scp cloud_classifier.onnx cloud_classifier_inference.py ronny@192.168.1.87:/home/ronny/emsn2/scripts/atmosbird/

# Installeer dependencies op Pi
pip install onnxruntime pillow numpy

# Test
cd /home/ronny/emsn2/scripts/atmosbird/
python cloud_classifier_inference.py cloud_classifier.onnx /mnt/usb/atmosbird/ruwe_foto/2025/12/30/sky_20251230_120000.jpg
```